In [9]:
!nvidia-smi

Tue May 21 07:12:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [11]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5597.6/8062.4 GB disk)


In [12]:
from ultralytics import YOLO

from IPython.display import display, Image

In [19]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/kaggle/working/datasets


In [20]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="1gBpoxVirzj0NHBuskuN")
project = rf.workspace("chaconneluofire").project("fire-smoke-firesmall")
version = project.version(1)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 28.4 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12.1
  Attempting uninstall: certifi
    Found existing install


Extracting Dataset Version Zip to Fire-Smoke-FireSmall-1 in yolov8:: 100%|██████████| 5866/5866 [00:00<00:00, 7162.04it/s]


In [21]:
%cd {HOME}

!yolo train model=yolov8s.pt data={dataset.location}/data.yaml epochs=100 imgsz=640 plots=True device=0,1 format=onnx batch=128 

/kaggle/working
New https://pypi.org/project/ultralytics/8.2.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                       CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/datasets/Fire-Smoke-FireSmall-1/data.yaml, epochs=100, patience=50, batch=128, imgsz=640, save=True, save_period=-1, cache=False, device=(0, 1), workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=

In [22]:
!yolo export model=/kaggle/working/runs/detect/train2/weights/best.pt format=onnx
!yolo export model=/kaggle/working/runs/detect/train2/weights/last.pt format=onnx

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/kaggle/working/runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 10, 8400) (21.5 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 1.1s, saved as '/kaggle/working/runs/detect/train2/weights/best.onnx' (42.7 MB)

Export complete (3.5s)
Results saved to /kaggle/working/runs/detect/train2/weights
Predict:         yolo predict task=detect model=/kaggle/working/runs/detect/train2/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/kaggle/working/runs/detect/train2/weights/best.onnx imgsz=640 data=/kaggle/working/datasets/Fire-Smoke-1/data.yaml  
Visualize:       https://netron.app
💡 Learn more at https://docs.ultralytics.com/modes/export
Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CPU (Intel Xeon 2.

In [23]:
!tar -czvf yolov8-b128-e50-img640.tar.gz /kaggle/working/runs/detect/train2/

tar: Removing leading `/' from member names
/kaggle/working/runs/detect/train2/
/kaggle/working/runs/detect/train2/P_curve.png
/kaggle/working/runs/detect/train2/train_batch2.jpg
/kaggle/working/runs/detect/train2/train_batch1.jpg
/kaggle/working/runs/detect/train2/train_batch0.jpg
/kaggle/working/runs/detect/train2/PR_curve.png
/kaggle/working/runs/detect/train2/confusion_matrix_normalized.png
/kaggle/working/runs/detect/train2/labels_correlogram.jpg
/kaggle/working/runs/detect/train2/val_batch2_pred.jpg
/kaggle/working/runs/detect/train2/results.png
/kaggle/working/runs/detect/train2/labels.jpg
/kaggle/working/runs/detect/train2/results.csv
/kaggle/working/runs/detect/train2/train_batch800.jpg
/kaggle/working/runs/detect/train2/train_batch801.jpg
/kaggle/working/runs/detect/train2/val_batch1_labels.jpg
/kaggle/working/runs/detect/train2/args.yaml
/kaggle/working/runs/detect/train2/F1_curve.png
/kaggle/working/runs/detect/train2/val_batch2_labels.jpg
/kaggle/working/runs/detect/train2